In [1]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

pio.renderers.default = 'browser'  # To open plot in a browser

def compute_Xn(A, X0, n):
    A_n = np.linalg.matrix_power(A, n)
    Xn = np.dot(A_n, X0)
    return Xn

def calculate_eigenvalue_decomposition_Xn(A, X0, n):
    # Calculate eigenvalues and eigenvectors (handles complex values)
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Calculate initial coefficients based on the initial state vector X0
    alpha_0 = np.dot(np.linalg.inv(eigenvectors), X0)
    
    # Initialize the state vector for n steps
    Xn_eigen = np.zeros_like(X0, dtype=np.complex128)  # Use complex type for potential complex values
    
    for i in range(len(eigenvalues)):
        Xn_eigen += alpha_0[i] * (eigenvalues[i]**n) * eigenvectors[:, i]
    
    return Xn_eigen

def visualize_evolution_and_steady_state(A, X0, steps=100, tolerance=1e-4):
    Xn_over_time = np.zeros((steps, len(X0)), dtype=np.complex128)  # Ensure complex type here
    steady_state_step = None
    
    for n in range(steps):
        Xn = compute_Xn(A, X0, n)
        Xn_over_time[n] = Xn
        
        # Check for steady state
        if n > 0 and np.allclose(Xn_over_time[n], Xn_over_time[n-1], atol=tolerance):
            if steady_state_step is None:
                steady_state_step = n
    
    # Plot the evolution using Plotly
    traces = []
    for i in range(len(X0)):
        trace = go.Scatter(
            x=np.arange(steps),
            y=np.real(Xn_over_time[:, i]),  # Plot real part
            mode='lines+markers',
            name=f'Component {i+1}',
            hovertemplate=f'Component {i+1}: {{y}}<br>Step: {{x}}<extra></extra>',
        )
        traces.append(trace)
    
    layout = go.Layout(
        title="Evolution of X(n) Over Time",
        xaxis_title="Time Steps (n)",
        yaxis_title="X(n) Components (Real Part)",
        hovermode='closest',
        legend_title="Components"
    )
    
    fig = go.Figure(data=traces, layout=layout)
    fig.show()
    
    # Plot the steady-state distribution
    if steady_state_step is None:
        steady_state_step = steps - 1
    Xn_steady_state = Xn_over_time[steady_state_step]
    
    bar_trace = go.Bar(
        x=np.arange(1, len(X0) + 1),
        y=np.real(Xn_steady_state),  # Plot real part of steady-state
        hovertemplate='Component: %{x}<br>Value: %{y}<extra></extra>',
        marker_color='blue'
    )
    
    bar_layout = go.Layout(
        title=f"Steady-State Distribution of X(n) at step {steady_state_step}",
        xaxis_title="Component",
        yaxis_title="Steady-State Value (Real Part)",
        hovermode='closest',
        xaxis=dict(tickmode='linear')
    )
    
    bar_fig = go.Figure(data=[bar_trace], layout=bar_layout)
    bar_fig.show()
    
    return steady_state_step

def visualize_specific_steps(A, X0, steps_list):
    results = {}
    
    for n in steps_list:
        Xn = compute_Xn(A, X0, n)
        results[n] = Xn
    
    # Plot state vector X(n) at specific time steps using Plotly
    traces = []
    
    for n in steps_list:
        trace = go.Bar(
            x=np.arange(1, len(X0) + 1),
            y=np.real(results[n]),  # Plot real part
            name=f'n = {n}',
            hovertemplate=f'Step {n}: {{y}}<br>Component: {{x}}<extra></extra>',
        )
        traces.append(trace)
    
    layout = go.Layout(
        title="State Vector X(n) at Specific Time Steps",
        xaxis_title="Component",
        yaxis_title="Value (Real Part)",
        hovermode='closest',
        xaxis=dict(tickmode='linear')
    )
    
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

def get_initial_state_options(n):
    options = {}
    for i in range(n):
        vector = np.zeros(n)
        vector[i] = 1
        options[chr(65+i)] = vector
    return options

def main():
    print("Enter the transition matrix A row by row, with each element separated by spaces:")
    A = []
    while True:
        row = input("Enter row (or type 'done' to finish): ").strip()
        if row.lower() == 'done':
            break
        A.append([float(x) for x in row.split()])
    A = np.array(A)
    
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square.")
    
    n = A.shape[0]
    options = get_initial_state_options(n)
    
    print("\nWould you like to choose from predefined initial state vectors or input your own?")
    print("1. Choose from predefined options (unit vectors).")
    print("2. Input your own initial state vector.")
    choice = input("Enter 1 or 2: ").strip()

    if choice == '1':
        print("\nAvailable initial state vector options:")
        for key, value in options.items():
            print(f"{key}: {value}")
        vector_choice = input("Choose initial state vector (A, B, C, ...): ").strip().upper()
        if vector_choice not in options:
            raise ValueError("Invalid choice. Choose a valid option.")
        X0 = options[vector_choice]
    elif choice == '2':
        X0 = np.array([float(x) for x in input(f"Enter your own initial state vector (length {n}, values separated by spaces): ").split()])
        if len(X0) != n:
            raise ValueError(f"Invalid length! Initial state vector must have length {n}.")
    else:
        raise ValueError("Invalid choice. Choose either 1 or 2.")
    
    n_steps = int(input("Enter the number of time steps (n): ").strip())
    
    Xn_matrix_power = compute_Xn(A, X0, n_steps)
    Xn_eigen_decomposition = calculate_eigenvalue_decomposition_Xn(A, X0, n_steps)
    
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    steady_state_step = visualize_evolution_and_steady_state(A, X0, steps=n_steps)
    
    print("\n--- Results ---")
    print(f"Steady-state reached at step {steady_state_step}")
    print("X(n) using matrix power:", Xn_matrix_power)
    print("X(n) using eigenvalue decomposition:", Xn_eigen_decomposition)
    print("Eigenvalues:", eigenvalues)
    print("Eigenvectors:\n", eigenvectors)
    
    specific_n_values = [1, 2, 3, 4, 5, 6, n_steps]
    visualize_specific_steps(A, X0, specific_n_values)

if __name__ == "__main__":
    main()

Enter the transition matrix A row by row, with each element separated by spaces:
Enter row (or type 'done' to finish): 0.7 0.5 0.3
Enter row (or type 'done' to finish): 0.2 0.3 0.3
Enter row (or type 'done' to finish): 0.1 0.2 0.4
Enter row (or type 'done' to finish): done

Would you like to choose from predefined initial state vectors or input your own?
1. Choose from predefined options (unit vectors).
2. Input your own initial state vector.
Enter 1 or 2: 1

Available initial state vector options:
A: [1. 0. 0.]
B: [0. 1. 0.]
C: [0. 0. 1.]
Choose initial state vector (A, B, C, ...): A
Enter the number of time steps (n): 50

--- Results ---
Steady-state reached at step 9
X(n) using matrix power: [0.58064516 0.24193548 0.17741935]
X(n) using eigenvalue decomposition: [0.58064516+0.j 0.24193548+0.j 0.17741935+0.j]
Eigenvalues: [1.         0.34142136 0.05857864]
Eigenvectors:
 [[-0.88841509 -0.79410449  0.47596315]
 [-0.37017295  0.23258782 -0.81251992]
 [-0.27146017  0.56151667  0.3365567